In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
HOPSWORKS_PROJECT_NAME = "ml_prod_pipline"

In [3]:
import os
from dotenv import load_dotenv
from src.paths import PARENT_DIR

# load key-val pairs from .env file located in the parent directory
load_dotenv(PARENT_DIR / ".env")

HOPSWORKS_API_KEY = os.environ["HOPSWORKS_API_KEY"]

In [4]:
from datetime import datetime
import pandas as pd
from src.data import loadRawData

from_year = 2022
to_year = datetime.now().year

print(f"Downloading raw data from {from_year} to {to_year}")

rides = pd.DataFrame()

for year in range(from_year, to_year + 1):
    
    # Download data for the whole year
    rides_one_year = loadRawData(year)
    
    # Append Rows
    rides = pd.concat([rides, rides_one_year])
    


File for 2022_01 found locally...
File for 2022_02 found locally...
File for 2022_03 found locally...
File for 2022_04 found locally...
File for 2022_05 found locally...
File for 2022_06 found locally...
File for 2022_07 found locally...
File for 2022_08 found locally...
File for 2022_09 found locally...
File for 2022_10 found locally...
File for 2022_11 found locally...
File for 2022_12 found locally...
File for 2023_01 found locally...
File for 2023_02 found locally...
File for 2023_03 not found, Downloading file...
File for 2023_04 not found, Downloading file...
File for 2023_04 not available
File for 2023_05 not found, Downloading file...
File for 2023_05 not available
File for 2023_06 not found, Downloading file...
File for 2023_06 not available
File for 2023_07 not found, Downloading file...
File for 2023_07 not available
File for 2023_08 not found, Downloading file...
File for 2023_08 not available
File for 2023_09 not found, Downloading file...
File for 2023_09 not available
Fi

In [5]:
print(f"{len(rides)=}")

len(rides)=49038833


In [6]:
from src.data import processRawData

ts_data = processRawData(rides)

100%|██████████| 262/262 [00:01<00:00, 159.04it/s]


In [7]:
ts_data


,pickup_hr,rides,pickup_loc_id
0,2022-01-01 00:00:00,11,4
1,2022-01-01 01:00:00,15,4
2,2022-01-01 02:00:00,26,4
3,2022-01-01 03:00:00,8,4
4,2022-01-01 04:00:00,9,4
...,...,...,...
2861297,2023-03-31 20:00:00,0,199
2861298,2023-03-31 21:00:00,0,199
2861299,2023-03-31 22:00:00,0,199
2861300,2023-03-31 23:00:00,0,199


In [8]:
ts_data
try:
    pd.to_datetime(ts_data['pickup_hr'], format='%d-%b-%Y', errors='raise')
    # do something
except ValueError:
    print("prob")
    pass

KeyError: 'pickup_hr'

In [ ]:
import hopsworks

project = hopsworks.login(
    project=HOPSWORKS_PROJECT_NAME,
    api_key_value=HOPSWORKS_API_KEY
)



Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/42132


In [ ]:
feature_store = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [ ]:
FEATURE_GROUP_NAME = "ts_hourly_feat_group"
FEATURE_GROUP_VERSION = 1

In [ ]:
feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ["pickup_loc_id", "pickup_hr"],
    event_time = "pickup_hr"
)

In [ ]:
feature_group.insert(ts_data, write_options={"wait_for_job": False} )

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/42132/fs/42052/fg/47673


Uploading Dataframe: 0.00% |          | Rows 0/2666374 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/42132/jobs/named/ts_hourly_feat_group_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7f134046afd0>, None)